In [3]:
from matplotlib import pyplot as plt
import os
import pandas as pd
import requests
import sys
import zipfile

%matplotlib inline

def check_download_extract(url):
    zip_name = url[url.rfind('/') + 1:]
    csv_name = zip_name[:zip_name.rfind('.zip')]
    if os.path.isfile(csv_name):
        print("{} found".format(csv_name))
    else:
        print("Downloading {}...".format(zip_name))
        r = requests.get(url)

        if os.path.isfile(zip_name):
            os.remove(zip_name)

        with open(zip_name, 'wb') as f:
            f.write(r.content)

        print("Extracting {} from {}...".format(csv_name, zip_name))
        with zipfile.ZipFile(zip_name) as z:
            z.extractall()

        if os.path.isfile(csv_name):
            os.remove(zip_name)
            print("{} successfully extracted\n".format(csv_name))
        else:
            print("Error getting csv from {}".format(url))
            sys.exit(1)

    return csv_name

In [2]:
loan_url = 'https://resources.lendingclub.com/LoanStats3a.csv.zip'
declined_url = 'https://resources.lendingclub.com/RejectStatsA.csv.zip'

# Download csv files
loan_csv = check_download_extract(loan_url)
declined_csv = check_download_extract(declined_url)

# Load loan data, skip first and last 2 rows
loan_df = pd.read_csv(loan_csv, skiprows=1, skipfooter=2, engine='python')

# Drop all columns that contain all null values or a single value

for col in loan_df:
    if loan_df[col].isnull().all() or len(loan_df[col].value_counts()) == 1:
        loan_df = loan_df.drop(col, axis=1)

# Load declined data, skip first row
declined_df = pd.read_csv(declined_csv, skiprows=1)

LoanStats3a.csv found
RejectStatsA.csv found
